# **Capítulo 9: Procesamiento de lenguaje natural**

## Traducción automática de texto: de español a inglés

Descargamos los datos del caso práctico

In [1]:
!wget -q http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
!unzip -q spa-eng.zip

In [2]:
# !pip install keras_nlp
# !pip install keras-nlp --upgrade
# !pip install -q git+https://github.com/keras-team/keras-nlp.git --upgrade
!pip install -q git+https://github.com/keras-team/keras-nlp.git --upgrade

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 851.9/851.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 90.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 110.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 41.5 MB/s eta 0:00:00


Carga del conjunto de datos de traducción automática

In [3]:
def cargar_datos():
    with open('spa-eng/spa.txt', 'r') as f:
        lineas = f.read().splitlines()
    pares = [linea.split('\t') for linea in lineas]
    esp = [par[1] for par in pares]
    ing = [par[0] for par in pares]
    return esp, ing

X, Y = cargar_datos()
print(f'Número de pares de oraciones: {len(X)}')
print(f'Posible entrada: {X[50]}')
print(f'Posible salida: {Y[50]}')

Número de pares de oraciones: 118964
Posible entrada: Estoy levantado.
Posible salida: I'm up.


Creación de los vocabularios de español e inglés

In [4]:
import re

def crear_vocab(frases):
    # Obtenemos el vocabulario
    vocab = set()
    for f in frases:
        # Expresión regular para separar palabras
        # manteniendo signos de puntuación
        vocab.update(re.findall(r'\w+|[^\w\s]', f))

    # Creamos los diccionarios
    w2i = {w: i+4 for i, w in enumerate(vocab)}
    w2i['PAD'] = 0
    w2i['SOS'] = 1
    w2i['EOS'] = 2
    w2i['UNK'] = 3
    i2w = {i: w for w, i in w2i.items()}

    return w2i, i2w

X_w2i, X_i2w = crear_vocab(X)
Y_w2i, Y_i2w = crear_vocab(Y)
print(f'Tamaño del vocabulario de español: {len(X_w2i)}')
print(f'Tamaño del vocabulario de inglés: {len(Y_w2i)}')

Tamaño del vocabulario de español: 28993
Tamaño del vocabulario de inglés: 14779


Codificación de las secuencias

In [5]:
def codificar(secs, w2i):
    secs_cod = []
    for s in secs:
        s_cod = [w2i[w] for w in re.findall(r'\w+|[^\w\s]', s)]
        s_cod = [w2i['SOS']] + s_cod + [w2i['EOS']]
        secs_cod.append(s_cod)
    return secs_cod

X_cod = codificar(X, X_w2i)
Y_cod = codificar(Y, Y_w2i)

División del conjunto de datos en entrenamiento y test (80-20)

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X_cod, Y_cod,\
                                                    test_size=0.2,\
                                                    random_state=42)
print('¡Particiones realizadas!')
print(f'Tamaño del conjunto de entrenamiento: {len(X_train)}')
print(f'Tamaño del conjunto de test: {len(X_test)}')

¡Particiones realizadas!
Tamaño del conjunto de entrenamiento: 95171
Tamaño del conjunto de test: 23793


Preprocesado de los datos de entrenamiento

In [7]:
import numpy as np

def preproceso_batch(X, Y):
    max_long_X = max([len(x) for x in X])
    max_long_Y = max([len(y) for y in Y])

    encoder_entrada = np.zeros((len(X), max_long_X))
    decoder_entrada = np.zeros((len(Y), max_long_Y))
    salida = np.zeros((len(Y), max_long_Y))

    for i, s in enumerate(X):
        # Sec. completa con relleno para el encoder (frase a traducir)
        encoder_entrada[i, :len(s)] = np.array(s)

    for i, s in enumerate(Y):
        # Sec. sin el "EOS" con relleno para el decoder (traducción)
        decoder_entrada[i, :len(s)-1] = np.array(s[:-1])
        # Sec. sin el "SOS" con relleno para la salida (traducción)
        salida[i, :len(s)-1] = np.array(s[1:])

    encoder_entrada = encoder_entrada.astype(np.int64)
    decoder_entrada = decoder_entrada.astype(np.int64)
    salida = salida.astype(np.int64)


    return encoder_entrada, decoder_entrada, salida


Creación de un generador de batches o data loader

In [8]:
from sklearn.utils import shuffle

def generador_batch(X, Y, batch_size):
    idx = 0
    while True:
        bx = X[idx:idx+batch_size]
        by = Y[idx:idx+batch_size]

        yield preproceso_batch(bx, by)

        idx = (idx + batch_size)
        if idx >= len(X):
            X, Y = shuffle(X, Y, random_state=42)
            return


tam_batch = 128
train_loader = generador_batch(X_train, Y_train, batch_size=tam_batch)
be, bd, bs = next(train_loader)
print(f'Entrada al encoder: {[X_i2w[w.item()]for w in be[0]]}')
print(f'Entrada al decoder: {[Y_i2w[w.item()]for w in bd[0]]}')
print(f'Salida del decoder: {[Y_i2w[w.item()]for w in bs[0]]}')

Entrada al encoder: ['SOS', 'No', 'tengo', 'otra', 'opción', 'en', 'absoluto', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD']
Entrada al decoder: ['SOS', 'I', 'have', 'no', 'choice', 'at', 'all', '.', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD']
Salida del decoder: ['I', 'have', 'no', 'choice', 'at', 'all', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD']


In [9]:
test_loader = generador_batch(X_test, Y_test, batch_size=tam_batch)

Definición de la capa de codificación de posición

In [10]:
import tensorflow as tf
import numpy as np

class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, max_len, emb_dim, dropout=0.1):
      super(PositionalEncoding, self).__init__()
      self.dropout = tf.keras.layers.Dropout(dropout)

      pos = np.arange(max_len).reshape(-1, 1)
      den = np.power(10000, np.arange(0, emb_dim, 2) / emb_dim)
      pe = np.zeros((1, max_len, emb_dim))
      pe[0, :, 0::2] = np.sin(pos / den)
      pe[0, :, 1::2] = np.cos(pos / den)
      self.pe = tf.constant(pe, dtype=tf.float32)

    def call(self, x):
      # x.shape = [batch_size, sec_len, emb_dim]
      x = x + self.pe[:, :tf.shape(x)[1], :]
      return self.dropout(x)


Definición del modelo Transformer a usar

In [11]:
import tensorflow as tf
import numpy as np
import keras_nlp

# Define la clase Transformer
class Transformer(tf.keras.Model):
  def __init__(self,
                max_long,
                emb_dim,
                num_enc_capas,
                num_dec_capas,
                ncabezas,
                src_vocab_tam,
                tgt_vocab_tam,
                dim_mlp,
                dropout=0.1):
      super(Transformer, self).__init__()

      # Capas de embedding + codificación de posición
      self.src_emb = tf.keras.layers.Embedding(src_vocab_tam, emb_dim)
      self.tgt_emb = tf.keras.layers.Embedding(tgt_vocab_tam, emb_dim)

      enc_entradas = tf.keras.Input(shape=(None,), dtype="int64", name="enc_entradas")
      enc_salidas = self.src_emb(enc_entradas)
      enc_salidas = PositionalEncoding(max_long, emb_dim, 0.1)(enc_salidas)

      # Encoder
      for i in range(num_enc_capas):
        enc_salidas = keras_nlp.layers.TransformerEncoder(
            intermediate_dim=dim_mlp,
            num_heads=ncabezas,
            dropout=dropout,
            activation="relu",
            name=None,
          )(enc_salidas)

      self.encoder = tf.keras.Model(enc_entradas, enc_salidas)

      # Decoder
      dec_entradas = tf.keras.Input(shape=(None,), dtype="int64", name="dec_entradas")
      dec_salidas = self.tgt_emb(dec_entradas)
      dec_salidas = PositionalEncoding(max_long, emb_dim, 0.1)(dec_salidas)

      enc_seq_entradas = keras.Input(shape=(None, emb_dim), name="dec_state_entradas")

      for _ in range(num_dec_capas):
        dec_salidas = keras_nlp.layers.TransformerDecoder(
            intermediate_dim=dim_mlp,
            num_heads=ncabezas,
            dropout=dropout,
            activation="relu",
            name=None,
          )(decoder_sequence=dec_salidas, encoder_sequence=enc_seq_entradas)

      dec_salidas = tf.keras.layers.Dense(tgt_vocab_tam, activation="linear")(dec_salidas)
      self.decoder = tf.keras.Model([
              dec_entradas, # input 1
              enc_seq_entradas, # input 2
          ],
          dec_salidas, # output
      )

      dec_salidas = self.decoder([dec_entradas, enc_salidas])
      self.transformer = tf.keras.Model(
        [enc_entradas, dec_entradas],
        dec_salidas,
        name="transformer",
      )

  def call(self, src, tgt):
      src_mask, _, _, _ = self.crear_mascara(src, tgt)

      # Transformer Encoder
      memory = self.encoder(src, mask=src_mask, training=True)


      # Transformer Decoder
      tgt_pred = self.decoder([tgt, memory], training=True)

      return tgt_pred

  def codificar(self, src, src_mask):
      # Embedding + codificación de posición
      return self.encoder(src, mask=src_mask, training=False)

  def decodificar(self, tgt, memory, tgt_mask):
      tgt_pred = self.decoder([tgt, memory], training=False)
      return tgt_pred

  def crear_mascara(self, src, tgt):
      # src/tgt.shape = [batch_size, src/tgt_sec_len, emb_dim]
      src_sec_len = tf.shape(src)[1]
      tgt_sec_len = tf.shape(tgt)[1]

      # Máscara de ceros (dejamos ver todo)
      src_mask = tf.zeros((src_sec_len, src_sec_len))
      # Máscara triangular superior para el target
      tgt_mask = tf.linalg.LinearOperatorLowerTriangular(tf.ones((tgt_sec_len, tgt_sec_len))).to_dense()

      # 0 == "PAD"
      src_pad_mask = (src == 0)
      tgt_pad_mask = (tgt == 0)

      return src_mask, tgt_mask, src_pad_mask, tgt_pad_mask


Using TensorFlow backend


In [12]:
max_long = max([len(x) for x in X + Y])
print(max_long)

278


Instancia del modelo Transformer a usar

In [13]:
# Instancia del modelo Transformer
modelo = Transformer(
    max_long=max_long,
    emb_dim=512,
    num_enc_capas=6,
    num_dec_capas=6,
    ncabezas=8,
    src_vocab_tam=len(X_w2i),
    tgt_vocab_tam=len(Y_w2i),
    dim_mlp=2048,
    dropout=0.1
)

In [14]:
# print(modelo.transformer.summary())
print(modelo.transformer.summary())


Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 enc_entradas (InputLayer)   [(None, None)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 512)            1484441   ['enc_entradas[0][0]']        
                                                          6                                       
                                                                                                  
 positional_encoding (Posit  (None, None, 512)            0         ['embedding[0][0]']           
 ionalEncoding)                                                                                   
                                                                                        

In [15]:
func_perdida = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    ignore_class=0, # Ignorar padding
)
optimizador = keras.optimizers.Adam(
    learning_rate=0.0001,
    beta_1=0.9,
    beta_2=0.98,
    epsilon=1e-9
)

epocas = 10
tam_batch = 128

for epoca in range(epocas):
    print(f'Comienzo de epoca {epoca}')
    train_loader = generador_batch(X_train, Y_train, batch_size=tam_batch)

    epoca_perdidas = []

    for step, (x, entrada_decoder, y) in enumerate(train_loader):

        # Objeto GradientTape para grabar las operaciones durante la pasada
        # forward y hacer la auto-diferenciacion
        with tf.GradientTape() as tape:
            # Forward del modelo Transformer
            logits = modelo(x, entrada_decoder)  # Logits for this minibatch
            # Calcular la perdida para el mini-batch
            loss_value = func_perdida(y, logits)

            epoca_perdidas.append(loss_value)

            if step % 100 == 0:
              res = tf.argmax(logits, axis=-1).numpy()
              print(f'Prediccion:    {res[:5]}')
              print(f'Ground truth: {y[:5]}')

        # Llamada a método .gradient para calcular los gradientes de los pesos
        # con respecto a la pérdida obtenida
        grads = tape.gradient(loss_value, modelo.trainable_weights)

        # Aplicar un paso del descenso del gradiente,
        # actualizando los pesos del modelo
        optimizador.apply_gradients(zip(grads, modelo.trainable_weights))

        # Log cada 100 steps.
        if step % 100 == 0:
            print(f'Pérdida media en el step {step}/{len(X_train)//tam_batch}: {sum(epoca_perdidas)/len(epoca_perdidas)}')

    print(f'Pérdida época {epoca}: {sum(epoca_perdidas)/len(epoca_perdidas)}')


Comienzo de epoca 0
Prediccion:    [[ 4153 14272 11265 11149  8712  8123 12662   663 10446 10131   663 11391
   5555 11149  4264 12816 12816 12816 11149   604  9433]
 [ 8123 11149   149  9433   849  8529  4534  9433  8520 12252   663 11510
   5353  5353  4264 12816 12816 12816  9433 12816 13860]
 [ 6612  2908 11341 14272 14272  9433 13304   663  8520 13304   663 10006
   8388  2818  4264 12816 12816 12816 11149   604 11149]
 [ 3339  6700 14272 14272  8123  9042  2345  8520 13304   663  5016 11149
   5353 11149 11149 12816 12816  9032 12816  8123  7510]
 [11149  2908 11149  8712  3748  5353 11510  8073 13304 11831  5016 14272
   2908  9433 11831 12816 12816 12816  3087 13152  2345]]
Ground truth: [[ 7867  6458 13888 11436 13927 14455 12166     2     0     0     0     0
      0     0     0     0     0     0     0     0     0]
 [ 7867 13366  9102 12853 14084 12166     2     0     0     0     0     0
      0     0     0     0     0     0     0     0     0]
 [ 7804  6635  1682  2016     2  

Pérdida media en el step 0/743: 9.592031478881836
Prediccion:    [[ 7867  2194  2194 12166 12166 12166 12166 12166 12166 12166 12166 12166
  12166 12166 12166 12166 12166 12166 12166 12166     2]
 [ 7867  2194  2194 12166 12166 12166 12166 12166     2 12166 12166 12166
  12166 12166 12166 12166 12166     2 12166     2     2]
 [ 7867  2194  2194 12166 12166 12166 12166 12166 12166 12166 12166 12166
      2 12166 12166 12166 12166 12166 12166     2     2]
 [ 7867  2194  2194 12166 12166 12166 12166 12166 12166 12166 12166 12166
  12166 12166 12166 12166 12166 12166 12166 12166 12166]
 [ 7867  2194  2194 12166 12166 12166 12166 12166     2     2 12166 12166
  12166 12166 12166 12166 12166     2     2     2     2]]
Ground truth: [[ 3651 13390  7866  2016     2     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0]
 [ 7526  2194  3712 12436  9102  3017  9092 12166     2     0     0     0
      0     0     0     0     0     0     0     0     0]
 [ 

Pérdida y optimizador del modelo

Entrenamiento de 15 épocas

In [16]:
import numpy as np
import tensorflow as tf

def decodificacion_voraz(modelo, src, src_mask, max_len, tgt_w2i, tgt_i2w):
    # Codificación
    src_cod = modelo.codificar(src, src_mask)

    # Decodificación
    tgt_token = tf.constant([[tgt_w2i['SOS']]], dtype=tf.int64)

    print(f'Tgt token shape: {tgt_token.shape}')

    tgt_pred_decod = []
    for i in range(max_len):
        # Predicción del modelo
        tgt_mask = modelo.crear_mascara(tgt_token, tgt_token)[1]
        tgt_pred = modelo.decodificar(tgt_token, src_cod, tgt_mask)
        tgt_pred = tgt_pred[:, -1, :]  # Último token

        # Nos quedamos con el token más probable
        print(f'{tf.argmax(tgt_pred, axis=-1).numpy()[0]}')
        tgt_pred = tf.argmax(tgt_pred, axis=-1).numpy()[0]
        tgt_pred_decod.append(tgt_i2w[tgt_pred])

        print(f'token predicho: {tgt_pred}')
        print(f'secuencia: {tgt_pred_decod}')

        # Preparamos la nueva entrada del decoder
        tgt_token = np.hstack((tgt_token, np.array([[tgt_pred]])))

        # Comprobamos si se ha predicho el token de fin de secuencia
        if tgt_pred_decod[-1] == 'EOS':
            break

    return tgt_pred_decod


In [17]:
import numpy as np
import tensorflow as tf

def traducir(modelo, src_frase, src_w2i, tgt_w2i, tgt_i2w):
    # Codificamos la secuencia de entrada
    src_cod = codificar([src_frase], src_w2i)
    src_cod = tf.convert_to_tensor(src_cod, dtype=tf.int64)
    # src_cod = tf.expand_dims(src_cod, axis=0)  # Agregamos dimensión de batch [1, sec_len]

    # Máscara de ceros para el source (dejamos ver todo)
    src_mask = tf.zeros((src_cod.shape[1], src_cod.shape[1]))

    # Permitimos hasta 5 tokens más en la traducción
    max_len = src_cod.shape[1] + 5

    # Iniciamos la traducción
    tgt_pred_decod = decodificacion_voraz(modelo, src_cod, src_mask, max_len, tgt_w2i, tgt_i2w)

    # Quitamos los tokens de inicio y fin de secuencia
    tgt_pred_decod = [t for t in tgt_pred_decod if t not in ['SOS', 'EOS']]
    return ' '.join(tgt_pred_decod)


In [19]:
src_frase = 'Espero que te haya gustado el libro'
tgt_frase = traducir(
    modelo,
    src_frase,
    X_w2i,
    Y_w2i, Y_i2w,
)
print(f'Original: {src_frase}\nTraducción: {tgt_frase}')

Tgt token shape: (1, 1)
7867
token predicho: 7867
secuencia: ['I']
9883
token predicho: 9883
secuencia: ['I', 'hope']
10039
token predicho: 10039
secuencia: ['I', 'hope', 'you']
2194
token predicho: 2194
secuencia: ['I', 'hope', 'you', "'"]
12541
token predicho: 12541
secuencia: ['I', 'hope', 'you', "'", 'll']
7694
token predicho: 7694
secuencia: ['I', 'hope', 'you', "'", 'll', 'be']
8743
token predicho: 8743
secuencia: ['I', 'hope', 'you', "'", 'll', 'be', 'able']
9102
token predicho: 9102
secuencia: ['I', 'hope', 'you', "'", 'll', 'be', 'able', 'to']
2882
token predicho: 2882
secuencia: ['I', 'hope', 'you', "'", 'll', 'be', 'able', 'to', 'get']
3017
token predicho: 3017
secuencia: ['I', 'hope', 'you', "'", 'll', 'be', 'able', 'to', 'get', 'the']
6729
token predicho: 6729
secuencia: ['I', 'hope', 'you', "'", 'll', 'be', 'able', 'to', 'get', 'the', 'same']
1342
token predicho: 1342
secuencia: ['I', 'hope', 'you', "'", 'll', 'be', 'able', 'to', 'get', 'the', 'same', 'language']
12166
to